In [ ]:
%pip install requests pandas seaborn matplotlib plotly nbformat scipy --upgrade

In [ ]:
import requests # pip install requests    
import pandas as pd # pip install pandas
import seaborn as sns # pip install seaborn
import matplotlib.pyplot as plt # pip install matplotlib
import plotly.express as px  # pip install plotly

from pprint import pprint
import datetime
from IPython.display import clear_output
import time 
import os

In [ ]:

# set base name for the file
base_name = << your game here >>

extended_file = f'reviews_{base_name}_extended.csv'

# if needed make a new folder of the base name
if not os.path.exists(base_name):
   print(f'Missing folder {base_name}')
else:
    # prepend that folder to all file names

    extended_file = f'{base_name}/{extended_file}'


In [ ]:
# Load the fully processed CSV file
df = pd.read_csv(extended_file)

# Display basic information about the dataset
print(df.info())

In [ ]:
# add 2 columns to the dataframe one for the number of sentences and one for avg. number of words per sentence
df['num_sentences'] = df['review'].apply(lambda x: len(x.rstrip(". ").split(".")))
df['avg_words_per_sentence'] = df['review'].apply(lambda x: sum([len(s.split()) for s in x.rstrip(". ").split(".")]) / len(x.rstrip(". ").split(".")))

display(df.head())


In [ ]:
# Display the first few rows of the dataset
display(df.head())

In [ ]:
# Display summary statistics of numeric data in the dataset
# CH note that all reviews now are listed as unique i.e. no duplicates
display(df.describe(include='all'))

In [ ]:
# Check for missing values
print(df.isnull().sum())

In [ ]:
# CH print out the timespan of the reviews
# this is useful for dweciding what timespand to aggregate to
print(df['timestamp_created'].min(), df['timestamp_created'].max())

### Aggregate over time

In [ ]:
# CH aggregate means of values over time (monthly)
df_agg  = df.copy() 
df_agg['timestamp_created'] = pd.to_datetime(df_agg['timestamp_created']) # make python datetime object
df_agg.set_index('timestamp_created', inplace=True) # set index to timestamp_created

# drop review column as it is not numeric
df_agg.drop(columns=['review'], inplace=True)

# resample all numeric columns to biweekly
#df_agg = df_agg.resample('14D').mean() # resample all numeric columns to monthly

# resample all numeric columns to monthly
df_agg = df_agg.resample('MS').mean() # resample all numeric columns to monthly

# resample all numeric columns to 3 month
#df_agg = df_agg.resample('3MS').mean() # resample all numeric columns to 6 month

# resample all numeric columns to 6 month
#df_agg = df_agg.resample('6MS').mean() # resample all numeric columns to 6 month

df_agg.reset_index(inplace=True) # reset index to make timestamp_created a column again
df_agg['timestamp_created'] = df_agg['timestamp_created'].dt.strftime('%Y-%m') # format timestamp_created
display(df_agg)



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px


def plot_over_time_sns(columns, df, logy=False):
    df = df[columns + ['timestamp_created']]
    # Melting the DataFrame
    df_melted = df.melt('timestamp_created', var_name='variable', value_name='value')
    plt.figure(figsize=(20, 8))
    
    sns.lineplot(data=df_melted, x='timestamp_created', y='value', hue='variable')
    plt.xlim(df['timestamp_created'].min(), df['timestamp_created'].max())
    if logy:
        plt.yscale('log')

def plot_over_time_plotly(columns, df, logy=False):
    df = df[columns + ['timestamp_created']]
    # Melting the DataFrame
    df_melted = df.melt('timestamp_created', var_name='variable', value_name='value')
        # Assuming df_melted is already created and contains 'timestamp_created', 'variable', and 'value' columns
    fig = px.line(df_melted, x='timestamp_created', y='value', color='variable', log_y=logy)

    # Update layout if necessary, for example, setting the x-axis range
    fig.update_layout(xaxis_range=[df_melted['timestamp_created'].min(), df_melted['timestamp_created'].max()])

    if logy:
        fig.update_yaxes(type='log')

    fig.show()

### Plot aggregated values over time using variables that are similar

In [ ]:
plot_over_time_plotly(['positive_score', 'negative_score', 'objective_score'], df_agg, logy=False)

In [ ]:
plot_over_time_plotly(['word_count', 'adj_count', 'verb_count', 'noun_count', 'adv_count'], df_agg, logy=False)

In [ ]:
plot_over_time_plotly(['author_vote', 'other_votes', 'votes_funny', 'comment_count'], df_agg, logy=True)

- seems may 2017 had a lot of fun :)  (16 million votes!)

### Histograms

In [ ]:
# Distribution of Numerical Variables
# Visualize the distribution of numerical variables using histograms and boxplots.
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Histogram for numerical columns

dfs = df[['positive_score', 'negative_score', 'objective_score']]
dfs.hist(bins=100, figsize=(20, 10))
plt.show()

# Boxplot for numerical columns
#plt.figure(figsize=(15, 10))
#sns.boxplot(data=df)
#plt.xticks(rotation=90)
#plt.show()

In [ ]:
# plot a histogram of a column with a specific x limit
def histo_xlim(column, xlimit):
    df[column].hist(bins=200, figsize=(10, 3))
    plt.title(f'Histogram of {column}')
    plt.xlim([0, xlimit])  # Limit the x-axis to 500
    plt.show()

histo_xlim('word_count', 300)
histo_xlim('adj_count', 50)
histo_xlim('verb_count', 200)
histo_xlim('noun_count', 200)
histo_xlim('adv_count', 50)

### Percentages of adj, verb, noun, adv to word count

In [ ]:
# add columns to df that describe the ratio of adj, verb, noun, adv to word count
# i.e. this is the percentage of each type of word in the review


df['adj percentage'] = df['adj_count'] / df['word_count'] * 100
df['verb percentage'] = df['verb_count'] / df['word_count'] * 100
df['noun percentage'] = df['noun_count'] / df['word_count'] * 100
df['adv percentage'] = df['adv_count'] / df['word_count']  * 100

dfr = df[['adj percentage', 'verb percentage', 'noun percentage', 'adv percentage']]
axes = dfr.hist(bins=100, figsize=(10, 5))

# get the mean of the percentage of adj, verb, noun, adv
for c in ['adj percentage', 'verb percentage', 'noun percentage', 'adv percentage']:
    print(f"{c} mean: {df[c].mean()}")

# how many reviews have < 5% percentage of adj, verb, noun, adv ?
print("\nReviews with < 1% percentage of adj, verb, noun, adv")
for c in ['adj percentage', 'verb percentage', 'noun percentage', 'adv percentage']:
    print(f"{c} < 1%: {len(df[df[c] < 1])}")

### Fun with combinations of percentages

In [ ]:
# find the row indices of the review with the lowest percentage of adj, verb, noun, adv and print the review
for c in ['adj percentage', 'verb percentage', 'noun percentage', 'adv percentage']:
    print(f"Review with lowest {c}: {min(df[c])}")
    print(df.loc[df[c].idxmin()]['review'])
    print("\n")

In [ ]:
# find the row indices of the review with the higher percentage of adj, verb, noun, adv and print the review and the percent
for c in ['adj percentage', 'verb percentage', 'noun percentage', 'adv percentage']:
    print(f"Review with highest {c}: {max(df[c])}")
    print(df.loc[df[c].idxmax()]['review'])
    print("\n")

### Find reviews with a certain mix of percentages
- the below mix was given by CoPilot as for advanced reading ability (check?)

In [ ]:
# pull out row indices of reviews that have:
# 1) 28 - 32 % nouns and
# 2) 8 - 12% verbs and
# 3) 8 - 12% adjectives and 
# 4) 1 - 5% adverbs

good_noun_perc = set(df[(df['noun percentage'] >= 28) & (df['noun percentage'] <= 32)].index.tolist())
good_verb_perc = set(df[(df['verb percentage'] >= 8) & (df['verb percentage'] <= 12)].index.tolist())
good_adj_perc = set(df[(df['adj percentage'] >= 8) & (df['adj percentage'] <= 12)].index.tolist())
good_adv_perc = set(df[(df['adv percentage'] >= 1) & (df['adv percentage'] <= 5)].index.tolist())

# print length of sets
print(f"good_noun_perc: {len(good_noun_perc)}")
print(f"good_verb_perc: {len(good_verb_perc)}")
print(f"good_adj_perc: {len(good_adj_perc)}")
print(f"good_adv_perc: {len(good_adv_perc)}")


### find reviews that cover 2 or 3 of the 4 categories as good

In [ ]:
from itertools import combinations

# List of all sets
sets = [good_noun_perc, good_verb_perc, good_adj_perc, good_adv_perc]
set_names = ['good_noun_perc', 'good_verb_perc', 'good_adj_perc', 'good_adv_perc']

# Generate all 2-set combinations and print their intersection sizes
for (set1, set2), (name1, name2) in zip(combinations(sets, 2), combinations(set_names, 2)):
    intersection_length = len(set1 & set2)
    print(f"Intersection length of {name1} and {name2}: {intersection_length}")

# Generate and print 3-set combinations and their intersection sizes
for (set1, set2, set3), (name1, name2, name3) in zip(combinations(sets, 3), combinations(set_names, 3)):
    intersection_length = len(set1 & set2 & set3)
    print(f"Intersection length of {name1}, {name2}, and {name3}: {intersection_length}")

In [ ]:
# find index for Intersection length of good_noun_perc and good_verb_perc
good_verb_noun_adv = good_verb_perc & good_noun_perc  
print(f"good_verb_adj_adv: {len(good_verb_noun_adv)}")

# print reviews of good_verb_adj_adv 
for i in good_verb_noun_adv:
    print(df.loc[i]['review'])
    print("\n\n")

In [ ]:
# find index for Intersection length of good_verb_perc, good_adj_perc, and good_adv_perc
good_verb_adj_adv = good_verb_perc & good_adj_perc & good_adv_perc
print(f"good_verb_adj_adv: {len(good_verb_adj_adv)}")

# print reviews of good_verb_adj_adv 
for i in good_verb_adj_adv:
    print(df.loc[i]['review'])
    print("\n")

#### How many reviews have nouns as the most use type of words, etc. ?

In [ ]:
# to df add a column that indicates which of adj, verb, noun, adv is the biggest 
# encode with "adj_count", "verb_count", "noun_count", "adv_count"
df['POS percentage'] = df[['adj percentage', 'verb percentage', 'noun percentage', 'adv percentage']].idxmax(axis=1)

# show the distribution of the POS percentage
df['POS percentage'].value_counts().plot(kind='bar');

In [ ]:
# how row ids for reviews  with the highest percentage for adj and adv
print("highest type of words is adj")
print(df.loc[df['adj percentage'].idxmax()]['review'])
print("\nhighest type of words is adj")
print(df.loc[df['adv percentage'].idxmax()]['review'])


### KDE plots
- 2D histograms

In [ ]:
# Plotting KDE plots
plt.figure(figsize=(14, 10))

# KDE plot for positive sentiment score vs adjective count
sns.kdeplot(data=df, x='adj_count', y='positive_score', fill=True, cmap="Blues", thresh=0.05)
plt.title('KDE Plot: Adjective Count vs Positive Sentiment Score')
plt.xlabel('Adjective Count')
plt.ylabel('Positive Sentiment Score')
plt.xlim(0, 50) # limit x-axis to 0-600
plt.ylim(0, 0.2)
plt.show()


In [ ]:


# KDE plot for negative sentiment score vs adjective count
plt.figure(figsize=(14, 10))
sns.kdeplot(data=df, x='adj_count', y='negative_score', fill=True, cmap="Reds", thresh=0.05)
plt.title('KDE Plot: Adjective Count vs Negative Sentiment Score')
plt.xlabel('Adjective Count')
plt.ylabel('Negative Sentiment Score')
#plt.xlim(0, 600)
#plt.ylim(0, 0.2)
plt.show()


In [ ]:

# KDE plot for objective sentiment score vs adjective count
plt.figure(figsize=(14, 10))
sns.kdeplot(data=df, x='adj_count', y='objective_score', fill=True, cmap="Greens", thresh=0.05)
plt.title('KDE Plot: Adjective Count vs Objective Sentiment Score')
plt.xlabel('Adjective Count')
plt.ylabel('Objective Sentiment Score')
plt.xlim(0, 700)
plt.show()

In [ ]:

# KDE plot for objective sentiment score vs adjective count
plt.figure(figsize=(14, 10))
sns.kdeplot(data=df, x='adj_count', y='objective_score', fill=True, cmap="Greens", thresh=0.05)
plt.title('KDE Plot: Adjective Count vs Objective Sentiment Score')
plt.xlabel('Adjective Count')
plt.ylabel('Objective Sentiment Score')
plt.xlim(0, 700)
plt.show()

# Start here for correlations!

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import warnings

# Ignore UserWarning about figure layout
warnings.filterwarnings("ignore", category=UserWarning, module='seaborn.axisgrid')


### Correlation coeffcient

In [ ]:
# This is from 574 Lecture 41-Data Science
# visualizing the correlation coefficient
import pandas as pd
from scipy.stats import spearmanr

numeric_df = df.select_dtypes(include=['float64', 'int64'])

r2 = numeric_df.corr(method='pearson')   # pearson is the standard method of calculation the goodness of fit
display(r2)

In [ ]:
# list the correlation pairs that have the 10 absolute highest correlation values
# the diagonal is always 1.0
s = r2.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
so = so[so != 1.0]  # remove diagonal
so = so[::2]  # remove duplicates
#so = so[0:20]  # get the top 6
pd.set_option('display.max_rows', None)

# Split the Series into positive and negative correlations
positive_correlations = so[so > 0]
negative_correlations = so[so < 0]

# Print positive correlations from 1.0 to 0
print("Positive Correlations from 1.0 to 0:")
print(positive_correlations)  # This reverses the positive correlations if needed

# Print negative correlations from -1.0 to 0
print("\nNegative Correlations from -1.0 to 0:")
print(negative_correlations[::-1])


In [ ]:
numeric_df = df.select_dtypes(include=['float64', 'int64'])
r2s = numeric_df.corr(method='spearman')   # spearman is a rank-ordering method
display(r2s)

In [ ]:
# same but for spearman rank corr.
s = r2s.unstack()
so = s.sort_values(kind="quicksort", ascending=False)
so = so[so != 1.0]  # remove diagonal
so = so[::2]  # remove duplicates
#so = so[0:20]  # get the top 6
pd.set_option('display.max_rows', None)

# Split the Series into positive and negative correlations
positive_correlations = so[so > 0]
negative_correlations = so[so < 0]

# Print positive correlations from 1.0 to 0
print("Positive Correlations from 1.0 to 0:")
print(positive_correlations)  # This reverses the positive correlations if needed

# Print negative correlations from -1.0 to 0
print("\nNegative Correlations from -1.0 to 0:")
print(negative_correlations[::-1])

In [ ]:
# next step: correlation matrix between all values. However, first I need to get the R2s.
ax1, fig =  plt.subplots(figsize= (15, 15))
sns.heatmap(abs(r2), 
            annot = True, 
            fmt=".1f"); # show numbers, but with 1 digit only

### Scatter plots

In [ ]:
# Based on the 574 Lecture 41-Data Science
# scatterplot of votes_funny vs positive_score with regression line
ax1, fig =  plt.subplots(figsize= (10, 10))
plt.close(plt.gcf()) # I'm unclear why this is needed here but whatevs

x = "num_sentences"
y = "word_count"


p = sns.lmplot(data=df, x=x, y=y, height=10, aspect=1); 

#limit the x and y axes
p.set(xlim=(0, 200), ylim=(0, 1000));
#p.set(xscale="linear", yscale="log");
#p.set_axis_labels("Number of Votes funny", "Number of upvotes by others");
#p.set(title='Votes_funny vs positive_score with regression line');

- CH: there definitifly an outlier!
- also: x should be log

In [ ]:
# Based on the 574 Lecture 41-Data Science
# scatterplot of votes_funny vs positive_score with regression line
ax1, fig =  plt.subplots(figsize= (10, 10))
plt.close(plt.gcf()) # I'm unclear why this is needed here but whatevs

x = "num_sentences"
y = "word_count"

# make a df with no outliers
dfnu = df[(df[x] < 10000)]

p = sns.lmplot(data=dfnu, x=x, y=y, height=10, aspect=1); 
p.set(xscale="log", yscale="linear");
#p.set_axis_labels("Number of Votes funny", "Number of upvotes by others");
#p.set(title='Votes_funny vs positive_score with regression line');


In [ ]:
# Based on the 574 Lecture 41-Data Science
# scatterplot of votes_funny vs positive_score


fig, ax = plt.subplots(figsize= (10, 10))

x = "other_votes"
y = "weighted_vote_score"

sns.scatterplot(data=df, x=x, y=y, ax=ax)
ax.set(title=f'{x} vs {y}', xlabel=x, ylabel=y);

# set both axis to log
ax.set_xscale('log');
#ax.set_yscale('log');


- CH: given that r2 is based on a linear relationship and this is more of a log relationship, no wonder r2 is pretty low!

### Create interactive scatterplots with plotly
- plotly can show values on mouse-over so we could look at the review for it but those can be very long and are displayed in one line by default

In [ ]:
# Create a shortened multiline version of the review text
# as plotly is HTML based we need o use <br> for line breaks, not \n

multiline_reviews = []  # Initialize the list here to ensure it's empty before starting the loop

for review in df['review']:
    words = review.split()
    review_lines = []
    for i in range(0, len(words), 20):
        review_lines.append(' '.join(words[i:i+20]))
    review_lines = review_lines[:10]  # Limit to 10 lines
    multiline_reviews.append('<br>'.join(review_lines))

df['reviewML'] = multiline_reviews  # Now, the length should match the DataFrame's index

# show the first review
display(df['reviewML'].iloc[0])


- We can also use colors (red/green) to show if a review was up or downvoted by its reviewer
- We can leave the outlier in as we can zoom in with plotly!
- "other_votes" (how many other users upvoted this review?) vs weighted_vote_score (how many found it __helpful__?)

In [ ]:
# scatterplot using plotly
#x = "other_votes"
#y = "weighted_vote_score"

x = "num_sentences"
y = "word_count"

# add a column author_vote_cat that is "up" where author is 1 and "down" where author is 0
# this b/c for color_discrete_map we need a column with strings
df["author_vote_cat"] = df["author_vote"].map({1: "up", 0: "down"})

colr = "author_vote_cat"

fig = px.scatter(df, x=x, y=y, log_x=True, #log_y=True, 
                 #title='Votes_funny vs positive_score',
                 #labels={'votes_funny': 'Number of Votes funny', 'other_votes': 'Number of upvotes by others'},
                 hover_data="reviewML" ,    # add hover data
                 color=colr,  # column to use for color
                 color_discrete_map={"up":"green", "down":"red"}
                 )


fig.show()

how many user found the review funny vs how may users left any of the other voting types that are possible?

In [ ]:
x = "votes_funny"
y = "other_votes"

x = "avg_words_per_sentence"  
y = "num_sentences"

# add a column author_vote_cat that is "up" where author is 1 and "down" where author is 0
# this b/c for color_discrete_map we need a column with strings
df["author_vote_cat"] = df["author_vote"].map({1: "up", 0: "down"})

colr = "author_vote_cat"

# scatterplot using plotly
fig = px.scatter(df, x=x, y=y, log_x=False, log_y=False, 
                 #title='Votes_funny vs positive_score',
                 #labels={'votes_funny': 'Number of Votes funny', 'other_votes': 'Number of upvotes by others'},
                 hover_data="reviewML" ,    # add hover data
                 color=colr,  # column to use for color
                 color_discrete_map={"up":"green", "down":"red"}
                 )

fig.show()